Encoding of the initial dataset :

--

Dataset available there : http://37.187.140.181/tuxml_dataset/

Python v3.6.8

In [1]:
import numpy as np
import pandas as pd

Import

In [2]:
data = pd.read_table("dataset_before_encoding.csv", dtype = str, delimiter =',')
data.head()

,Unnamed: 0,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,nbyes
0,0,30000,2018-06-09 09:49:20,428.014,50222120,7603152.0,9689768,7403313,11789264.0,13875328,...,n,y,n,n,n,y,n,n,n,1435
1,1,30001,2018-06-09 09:49:41,241.167,16660024,4109776.0,6194296,3964503,8160720.0,10244688,...,n,y,y,n,n,m,m,m,n,1382
2,2,30002,2018-06-09 09:49:58,198.649,43080856,11191760.0,13276280,11046541,15177168.0,17261136,...,n,y,n,y,n,n,n,n,n,1626
3,3,30003,2018-06-09 09:50:05,214.03400000000002,27261672,8053712.0,10139088,7907068,11850704.0,13935528,...,y,y,y,y,n,y,n,y,n,2140
4,4,30004,2018-06-09 09:50:10,348.474,58769440,16135120.0,18220496,15986281,20018128.0,22102952,...,n,y,n,y,n,y,n,y,y,2651


Data processing :
- keep configurations with id > 30000
- keep kernel_size (here vmlinux) positive
- fill na
- remove other sizes
- keep only the yes/no/module features

In [3]:
data = data.fillna('na')


remove_columns = []

col = data.columns
for i in range(len(col)):
    name_col = col[i]
    actual_col = data[name_col]
    if len(actual_col.unique()) == 1:
        remove_columns.append(name_col)
    for j in range(len(actual_col)):
        if actual_col[j] not in ['na','y','n','m']:
            if name_col not in remove_columns:
                remove_columns.append(name_col)
remove_columns.remove('vmlinux')


remove_lines = []

cid = np.array(data["cid"], dtype = int)
for index_row in range(len(cid)):
    if cid[index_row] < 30000:
        remove_lines.append(index_row)

vmlinux = np.array(data["vmlinux"], dtype = int)
for index_row in range(len(vmlinux)):
    if vmlinux[index_row] < 0:
        if index_row not in remove_lines:
            remove_lines.append(index_row)


data_final = data.drop(remove_columns, axis = 1).drop(remove_lines, axis = 0)
data_final.head()

,vmlinux,OPENVSWITCH,TEXTSEARCH_FSM,NETFILTER_XT_MATCH_TCPMSS,MPLS,NFC_HCI,NETFILTER_XT_MATCH_TIME,NET_MPLS_GSO,NFC_SHDLC,NETFILTER_XT_MATCH_U32,...,VIDEO_S3C_CAMIF,SND_SOC_INTEL_SKL_NAU88L25_SSM4567_MACH,APDS9960,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS
0,50222120,n,n,n,y,n,n,y,n,n,...,n,n,n,n,n,n,y,n,n,n
1,16660024,n,n,n,n,n,n,n,n,n,...,n,n,n,y,n,n,m,m,m,n
2,43080856,n,n,n,n,n,n,n,n,n,...,n,n,n,n,y,n,n,n,n,n
3,27261672,n,n,n,n,n,n,n,n,n,...,n,n,y,y,y,n,y,n,y,n
4,58769440,n,n,n,n,y,n,n,y,n,...,n,n,n,n,y,n,y,n,y,y


Create the burt table related to the configurations

In [4]:
burt_table = pd.get_dummies(data_final.drop(['vmlinux'], axis = 1))
burt_table.head()
burt_table['vmlinux'] = data_final['vmlinux']
burt_table.to_csv("burt.csv")

In [5]:
burt_table.head()

,OPENVSWITCH_m,OPENVSWITCH_n,OPENVSWITCH_y,TEXTSEARCH_FSM_m,TEXTSEARCH_FSM_n,TEXTSEARCH_FSM_y,NETFILTER_XT_MATCH_TCPMSS_m,NETFILTER_XT_MATCH_TCPMSS_n,NETFILTER_XT_MATCH_TCPMSS_y,MPLS_n,...,CHARGER_BQ24190_y,SND_SOC_NAU8825_m,SND_SOC_NAU8825_n,SND_SOC_NAU8825_y,BH1750_m,BH1750_n,BH1750_y,NETWORK_FILESYSTEMS_n,NETWORK_FILESYSTEMS_y,vmlinux
0,0,1,0,0,1,0,0,1,0,0,...,1,0,1,0,0,1,0,1,0,50222120
1,0,1,0,0,1,0,0,1,0,1,...,0,1,0,0,1,0,0,1,0,16660024
2,0,1,0,0,1,0,0,1,0,1,...,0,0,1,0,0,1,0,1,0,43080856
3,0,1,0,0,1,0,0,1,0,1,...,1,0,1,0,0,0,1,1,0,27261672
4,0,1,0,0,1,0,0,1,0,1,...,1,0,1,0,0,0,1,0,1,58769440


Get dummies with yes and module

In [9]:
keep_yes_module = []
for col_name in burt_table.columns:
    if col_name[len(col_name)-1]!='n':
        keep_yes_module.append(col_name)
yes_mod = burt_table[keep_yes_module]
yes_mod.to_csv("burt_yes_mod.csv")